## Start With Gradio !!
We will build user interface usng Gradio framework.

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic


In [12]:
import gradio as gr

In [3]:
load_dotenv(override=True)
# Initialize OpenAI client
openai_api_key =os.getenv("OPENAI_API_KEY")
anthropic_api_key =os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

if openai_api_key:
    print("OpenAI API KEY Exists")
else:
    print("OpenAI API KEY does not exist, please set it in .env file")
if anthropic_api_key:
    print("Anthropic API KEY Exists")
else:
    print("Anthropic API KEY does not exist, please set it in .env file")
if google_api_key:
    print("Google API KEY Exists")  
else:
    print("Google API KEY does not exist, please set it in .env file")



OpenAI API KEY Exists
Anthropic API KEY does not exist, please set it in .env file
Google API KEY does not exist, please set it in .env file


In [4]:
# Initialize clients for each API

openai = OpenAI()

claude = anthropic.Anthropic()

google = google.generativeai.configure()


In [5]:
# System message
system_message = "You are a helpful AI assistant "

In [7]:
# Lets wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role":"system","content":system_message},
        {"role":"user","content":prompt}
    ]

    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return completion.choices[0].message.content.strip()


In [8]:
message_gpt("What is today date and day ?")

"Today's date is October 1, 2023, and it is a Sunday."

User Interface

In [9]:
# Create a function
def shout(text: str) -> str:
    return text.upper() + "!!!"

In [10]:
shout("Hello , World !!")

'HELLO , WORLD !!!!!'

In [15]:
gr.Interface(fn=shout,inputs="textbox",outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [16]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform using a platform called Spaces from HiggingFace.
gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://f06264dcc03db8efd1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [19]:
# inbrowser=True --> opens up a new browser window automatically
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Forcing dark mode :
Gradio appears in light mode or dark mode depending on the settings of the browser and computer.There is a way to force gradio to appear in dark mode, If you want to make dark moe for your screen.

In [20]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [23]:
# Inputs and ouputs

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your messaage:",lines=6)],
    outputs=[gr.Textbox(label="Shouted message:", lines=6)],
    flagging_mode="never",
)
view.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [26]:
## Lets change the functon from "shout" to "message_gpt"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=6)],
    flagging_mode="never",
)

view.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [ ]:
# Let see if use system_message 
# Taking advantage of system_message being a global variable, used back in the message_gpt function
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)

view.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [31]:
# Let's create a call that streams back results
def stream_gpt(prompt: str):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    
    stream = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=True
    )
    
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [32]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [33]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[{"role": "user", "content": prompt}],
    )

    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [ ]:
view = gr.Interface(
    fn=stream_claude,
    inputs = [gr.Textbox(label="Your message:")],
    outputs = [gr.Markdown(label="Response:")],
    flagging_mode="never"
)

view.launch()

# Minor Improvement
Small Improvement

Prevously, it had these lines:

for chunk in result:
  yield chunk

There is a more elegant way to achieve this 

yield from result


In [ ]:
def stream_model(prompt,model):
    if model == "GPT":
        print("Using GPT model")
        return stream_gpt(prompt)
    elif model == "Claude":
        return stream_claude(prompt)
    else:
        raise ValueError("Unsupported model. Please choose 'GPT' or 'Claude'.")
    yield from result

<generator object stream_model at 0x0000019E1E5026C0>

In [42]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


Using GPT model
